<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
##### Your Code Here #####
import pandas as pd
import numpy as np

data = {
    'x1': [0,0,1,0,1,1,0],
    'x2': [0,1,0,1,0,1,0],
    'x3': [1,1,1,0,0,1,0],
    'y' : [0,1,1,1,1,0,0]
}

df = pd.DataFrame.from_dict(data).astype('int')
df

,x1,x2,x3,y
0,0,0,1,0
1,0,1,1,1
2,1,0,1,1
3,0,1,0,1
4,1,0,0,1
5,1,1,1,0
6,0,0,0,0


In [2]:
# Neural Network Class from lecture
class NeuralNetwork:
    def __init__(self):
        # Set up Architecture of Neural Network
        self.inputs = 3
        self.hiddenNodes = 4
        self.outputNodes = 1

        # Initial Weights
        # 3x4 Matrix Array for the First Layer
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
       
        # 4x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        aka "predict"
        """
        
        # Weighted sum of inputs => hidden layer
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    def backward(self, X,y,o):
        """
        Backward propagate through the network
        """
        
        # Error in Output
        self.o_error = y - o
        
        # Apply Derivative of Sigmoid to error
        # How far off are we in relation to the Sigmoid f(x) of the output
        # ^- aka hidden => output
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        
        # z2 error
        self.z2_error = self.o_delta.dot(self.weights2.T)
        
        # How much of that "far off" can explained by the input => hidden
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        # Adjustment to first set of weights (input => hidden)
        self.weights1 += X.T.dot(self.z2_delta)
        # Adjustment to second set of weights (hidden => output)
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)
        

    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X,y,o)

In [3]:
#converting the matrixes to numpy arrays
#y is reshaped so that it is a column array, instead of the default of a row array.
X = df[['x1', 'x2', 'x3']].to_numpy()
y = df['y'].to_numpy().reshape(-1, 1)

In [4]:
# Train my 'net
nn = NeuralNetwork()

# Number of Epochs / Iterations
# For loop from lecture
for i in range(10000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 1000 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(nn.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X,y)

+---------EPOCH 1---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.82714668]
 [0.8452562 ]
 [0.85693724]
 [0.80612944]
 [0.82232522]
 [0.86989355]
 [0.78113547]]
Loss: 
 0.30923225392153725
+---------EPOCH 2---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.75860434]
 [0.77552102]
 [0.78673774]
 [0.74329485]
 [0.75662721]
 [0.79986833]
 [0.72212307]]
Loss: 
 0.2796758683885823
+---------EPOCH 3---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.67961753]
 [0.69253838]
 [0.70106367]
 [0.67280929]
 [0.68094324]
 [0.71192566]
 [0.6577786 ]]
Loss: 
 0.2563053408849458
+---------EPOCH 4---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [5]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [6]:
# input image dimensions
img_rows, img_cols = 28, 28

In [7]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [8]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [9]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [10]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [11]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

In [12]:
##### Your Code Here #####

# neural network class
class Neural_Network(object):
        def __init__(self, inputLayerSize=4, outputLayerSize=1, hiddenLayerSize=4):
            # Set up Architecture of Neural Network
            self.inputs = inputLayerSize
            self.hiddenNodes = outputLayerSize
            self.outputNodes = hiddenLayerSize
        
        #setting random weights for each layer of the newtork
            self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
            self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes)
        
        def sigmoid(self, s):
            return 1 / (1+np.exp(-s))

        def sigmoidPrime(self, s):
            return s * (1 - s)

        def feed_forward(self, X):
            """
            Calculate the NN inference using feed forward.
            aka "predict"
            """

            # Weighted sum of inputs => hidden layer
            self.hidden_sum = np.dot(X, self.weights1)

            # Activations of weighted sum
            self.activated_hidden = self.sigmoid(self.hidden_sum)

            # Weight sum between hidden and output
            self.output_sum = np.dot(self.activated_hidden, self.weights2)

            # Final activation of output
            self.activated_output = self.sigmoid(self.output_sum)

            return self.activated_output

        def backward(self, X,y,o):
            """
            Backward propagate through the network
            """

            # Error in Output
            self.o_error = y - o

            # Apply Derivative of Sigmoid to error
            # How far off are we in relation to the Sigmoid f(x) of the output
            # ^- aka hidden => output
            self.o_delta = self.o_error * self.sigmoidPrime(o)

            # z2 error
            # z2 error is how much are the weights of the hidden layer responsible for the error
            self.z2_error = self.o_delta.dot(self.weights2.T)
            # How much of that "far off" can explained by the input => hidden
            self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)

            # Adjustment to first set of weights (input => hidden)
            self.weights1 += X.T.dot(self.z2_delta)
            # Adjustment to second set of weights (hidden => output)
            self.weights2 += self.activated_hidden.T.dot(self.o_delta)


        def train(self, X, y):
            o = self.feed_forward(X)
            self.backward(X,y,o)


In [13]:
# initializing the class
nn = Neural_Network(inputLayerSize=x_train.shape[1])

In [14]:
# displaying results
# using for loop from lecture
for i in range(1000):
    if (i+1 in [1,2,3]) or ((i+1) % 500 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', x_train)
        print('Actual Output: \n', y_train)
        print('Predicted Output: \n', str(nn.feed_forward(x_train)))
        print("Loss: \n", str(np.mean(np.square(y_train - nn.feed_forward(x_train)))))
    nn.train(x_train,y_train)

+---------EPOCH 1---------+
Input: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Actual Output: 
 [0. 1. 0. ... 0. 0. 0.]
Predicted Output: 
 [[0.7050698  0.69778131 0.54076245 0.56230361]
 [0.7050698  0.69778131 0.54076245 0.56230361]
 [0.7050698  0.69778131 0.54076245 0.56230361]
 ...
 [0.7050698  0.69778131 0.54076245 0.56230361]
 [0.7050698  0.69778131 0.54076245 0.56230361]
 [0.7050698  0.69778131 0.54076245 0.56230361]]


ValueError: operands could not be broadcast together with shapes (60000,) (60000,4) 

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?